In [ ]:
# Complete Case kNN
for i in range(N_test):
    if y_test[i] == complete_case_kNN_classifier(x_test[i,:],X_train,Y_train,O_train):
        accuracy_cc_kNN[it] += 1/N_test

if not (O_train==np.ones((N_train,d))).all():        
    # Mean Imputation then kNN  
    X_imputed = mean_impute(X_train, O_train)
    for i in range(N_test):
        if y_test[i] == complete_case_kNN_classifier(x_test[i,:],X_imputed,Y_train,np.ones((N_train,d))):
            accuracy_mi_kNN[it] += 1/N_test

    # c=0 Imputation then kNN
    X_imputed = constant_impute(X_train, O_train)
    for i in range(N_test):
        if y_test[i] == complete_case_kNN_classifier(x_test[i,:],X_imputed,Y_train,np.ones((N_train,d))):
            accuracy_ci_kNN[it] += 1/N_test

    # MissForest imputation using missingpy
    X_miss = np.zeros((N_train,d))
    X_miss += X_train
    for i in range(N_train):
        for j in range(d):
            if O_train[i,j]==0:
                X_miss[i,j] = 'nan'
    sys.stdout = open(os.devnull, 'w')
    X_imputed = imputer.fit_transform(X_miss)
    sys.stdout = sys.__stdout__
    for i in range(N_test):
        if y_test[i] == complete_case_kNN_classifier(x_test[i,:],X_imputed,Y_train,np.ones((N_train,d))):
            accuracy_mf_kNN[it] += 1/N_test

# Oracle HAM
for i in range(N_test):
    if y_test[i] == HAM_classifier(x_test[i,:], Omega_star, X_train, Y_train, O_train, alpha, betas, gammas):
        accuracy_oracle_HAM[it] += 1/N_test

# HAM
Omega_hat = []
Omega_hat = estimate_Omega_star(X_train, Y_train, O_train, alpha, betas, gammas)
for i in range(N_test):
    if y_test[i] == HAM_classifier(x_test[i,:], Omega_hat, X_train, Y_train, O_train, alpha, betas, gammas):
        accuracy_HAM[it] += 1/N_test        

# cvHAM
Omega_hat = []
r_optimal = estimate_params_CV(X_train, Y_train, O_train)
Omega_hat = estimate_Omega_star(X_train, Y_train, O_train, r_optimal[0],r_optimal[1],r_optimal[2])
for i in range(N_test):
    if y_test[i] == HAM_classifier(x_test[i,:], Omega_hat, X_train, Y_train, O_train, r_optimal[0],r_optimal[1],r_optimal[2]):
        accuracy_cvHAM[it] += 1/N_test    

if not (O_train==np.ones((N_train,d))).all():
    # Complete Case kNN
    for i in range(N_test):
        if y_test[i] == complete_case_kNN_classifier(x_test[i,:],X_train,Y_train,np.ones((N_train,d))):
            accuracy_full_kNN[it] += 1/N_test

    # Random Forest Imputation + kNN
    X_miss = np.zeros((N_train,d))
    X_miss += X_train
    for i in range(N_train):
        for j in range(d):
            if O_train[i,j]==0:
                X_miss[i,j] = 'nan'
    # Run the MICE algorithm for 2 iterations and return 5 completed datasets
    kds = mf.ImputationKernel(X_miss, datasets=5)
    kds.mice(2)
    for i in range(N_test):
        y_s=0
        for s in range(5):
            X_train_t = kds.complete_data(dataset=s)
            y_s += complete_case_kNN_classifier(x_test[i,:],X_train_t,Y_train,np.ones((N_train,d)))/5
        if y_s>=1/2:
            y_s = 1
        else:
            y_s = 0
        if y_test[i] == y_s:
            accuracy_mice_kNN[it] += 1/N_test